In [52]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
import xgboost as xgb
from collections import Counter
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [53]:

df = pd.read_csv("/Users/josetudela/Projects/EligeTuVino/eligetuvino/data/datasets/raw/0_VERSION_FINAL_ESPU.csv")

In [54]:
pd.set_option("display.max_colwidth", None)

In [55]:
df["Precio"] = df["Precio"].fillna("").astype(str).str.replace(",", "", regex=True)


In [56]:
df["Precio"] = pd.to_numeric(df["Precio"], errors="coerce")

In [57]:
def limpiar_caza(texto):
    if isinstance(texto, str) and "caza" in texto.lower():  # Verifica si es texto y contiene "caza"
        return pd.Series(texto).replace(r"\s*\(.*?\)", "", regex=True).iloc[0]
    return texto  # Devuelve el texto sin cambios si no tiene "caza"

# Aplicar la limpieza a todas las celdas del DataFrame
df = df.applymap(limpiar_caza)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/3888707711.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_caza)


In [58]:
# Modificamos maridaje porque contiene una "," en medio de la descrpción 
def limpiar_pesca(texto):
    if isinstance(texto, str) and "azul " in texto.lower():  # Verifica si es texto y contiene "caza"
        return pd.Series(texto).replace(r"\s*\(.*?\)", "", regex=True).iloc[0]
    return texto  # Devuelve el texto sin cambios si no tiene "caza"

# Aplicar la limpieza a todas las celdas del DataFrame
df = df.applymap(limpiar_pesca)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/979100320.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(limpiar_pesca)


In [59]:
df["Maridajes"]

0                Marisco, Aves, Aperitivos y tentempiés, Pescado blanco, Aperitivo
1                Marisco, Aves, Aperitivos y tentempiés, Pescado blanco, Aperitivo
2                                      Marisco, Aperitivos y tentempiés, Aperitivo
3       Marisco, Aperitivos y tentempiés, Pescado blanco, Aperitivo, Carne adobada
4                                               Aperitivos y tentempiés, Aperitivo
                                           ...                                    
4150                          Cerdo, Marisco, Pescado azul, Queso tierno y cremoso
4151                          Cerdo, Marisco, Pescado azul, Queso tierno y cremoso
4152                          Cerdo, Marisco, Pescado azul, Queso tierno y cremoso
4153                          Cerdo, Marisco, Pescado azul, Queso tierno y cremoso
4154                          Cerdo, Marisco, Pescado azul, Queso tierno y cremoso
Name: Maridajes, Length: 4155, dtype: object

In [60]:
#Separamos la columna maridajes para que cada una sea una característica
df_maridaje_separado =  df["Maridajes"].str.split(',', expand=True)
df_maridaje_separado["ID"]= df["ID"]
df_maridaje_separado = df_maridaje_separado.fillna("Desconocido")
df_maridaje_separado[100:140]

,0,1,2,3,4,5,ID
100,Marisco,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,Desconocido,8763558
101,Marisco,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Carne adobada,Desconocido,1154695
102,Aperitivos y tentempiés,Aperitivo,Desconocido,Desconocido,Desconocido,Desconocido,10909946
103,Aperitivos y tentempiés,Aperitivo,Desconocido,Desconocido,Desconocido,Desconocido,1232522
104,Marisco,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,Desconocido,1533757
105,Marisco,Aves,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,1957556
106,Marisco,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Carne adobada,Desconocido,1399465
107,Marisco,Aves,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,11440485
108,Marisco,Aves,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,5906212
109,Marisco,Aves,Aperitivos y tentempiés,Pescado blanco,Aperitivo,Desconocido,2596568


In [61]:
valores_unicos_maridaje = set(df_maridaje_separado.drop(columns=["ID"]).values.ravel())
print(valores_unicos_maridaje)

{' Pescado blanco', 'Cerdo', ' Postres de frutas', 'Postres de frutas', 'Postres dulces', ' Aperitivos y tentempiés', 'Aperitivos y tentempiés', ' Aperitivo', 'Aves', ' Vegetariana', ' La comida basura vale', 'Marisco', ' Carne adobada', 'Queso curado', ' Pescado azul', ' Aves', ' Queso tierno y cremoso', ' Marisco', 'No disponible', 'Desconocido'}


In [62]:
df_maridaje_separado_sin_ID = df_maridaje_separado.drop(columns = "ID")

In [63]:
df_maridaje_separado_sin_ID = pd.get_dummies(df_maridaje_separado_sin_ID[[0,1]], prefix='', prefix_sep='')

In [64]:
df_maridaje_separado_sin_ID

,Aperitivos y tentempiés,Aves,Cerdo,Marisco,No disponible,Postres de frutas,Postres dulces,Queso curado,Aperitivo,Aperitivos y tentempiés,Aves,Carne adobada,La comida basura vale,Marisco,Postres de frutas,Vegetariana,Desconocido
0,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4151,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4152,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False
4153,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False


In [65]:
df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.drop(columns = ["Desconocido","No disponible"])

In [66]:
df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_maridaje_separado_sin_ID_encoded = pd.get_dummies(df_maridaje_separado_sin_ID,drop_first=True ,prefix="" ,prefix_sep="")

# Convertir a DataFrame
df_maridaje_separado_sin_ID_encoded = pd.DataFrame(df_maridaje_separado_sin_ID_encoded)

# Mostrar resultado
display(df_maridaje_separado_sin_ID_encoded)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/2076073784.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_maridaje_separado_sin_ID = df_maridaje_separado_sin_ID.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


,Aperitivos y tentempiés,Aves,Cerdo,Marisco,Postres de frutas,Postres dulces,Queso curado,Aperitivo,Aperitivos y tentempiés,Aves,Carne adobada,La comida basura vale,Marisco,Postres de frutas,Vegetariana
0,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
4151,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
4152,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
4153,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False


In [67]:
df_maridaje_separado_sin_ID_encoded.columns = df_maridaje_separado_sin_ID_encoded.columns.str.strip()

In [68]:
df_maridaje_separado_sin_ID_encoded=df_maridaje_separado_sin_ID_encoded.groupby(axis=1, level=0).sum()
df_maridaje_separado_encoded = df_maridaje_separado_sin_ID_encoded
df_maridaje_separado_encoded["ID"] = df["ID"]
df_maridaje_separado_encoded

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/1026340629.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_maridaje_separado_sin_ID_encoded=df_maridaje_separado_sin_ID_encoded.groupby(axis=1, level=0).sum()


,Aperitivo,Aperitivos y tentempiés,Aves,Carne adobada,Cerdo,La comida basura vale,Marisco,Postres de frutas,Postres dulces,Queso curado,Vegetariana,ID
0,0,0,1,0,0,0,1,0,0,0,0,3967508
1,0,0,1,0,0,0,1,0,0,0,0,4919479
2,0,1,0,0,0,0,1,0,0,0,0,6700139
3,0,1,0,0,0,0,1,0,0,0,0,4311421
4,1,1,0,0,0,0,0,0,0,0,0,5794981
...,...,...,...,...,...,...,...,...,...,...,...,...
4150,0,0,0,0,1,0,1,0,0,0,0,1101102
4151,0,0,0,0,1,0,1,0,0,0,0,1455336
4152,0,0,0,0,1,0,1,0,0,0,0,3979388
4153,0,0,0,0,1,0,1,0,0,0,0,11003457


In [69]:
df


,Url,ID,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,Amable/Con Burbujas,Taste_Notes_1_1,Taste_Notes_1_2,Taste_Notes_1_3,Taste_Notes_2_1,Taste_Notes_2_2,Taste_Notes_2_3,Taste_Notes_3_1,Taste_Notes_3_2,Taste_Notes_3_3
0,https://www.vivino.com/ES/es/fr-simonnet-febvre-cremant-de-bourgogne-cuvee-s-brut/w/3967508,3967508,Crémant de Bourgogne Cuvée S Brut,2018,Francia,Crémant de Bourgogne,Simonnet-Febvre,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,6.0,menta,pomelo,mora,cítrico,limón,pomelo,fresa madura,crema,queso
1,https://www.vivino.com/ES/es/louis-loron-cremant-de-bourgogne-blanc-tradition/w/4919479,4919479,Crémant de Bourgogne Blanc TraditionN.V.,No disponible,Francia,Crémant de Bourgogne,Louis Loron,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,6.5,crema,polvo de vainilla,polvo de vainilla,tabaco dulce,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla
2,https://www.vivino.com/ES/es/marcio-lopes-pequenos-rebentos-ancestral-loureiro-pet-nat/w/6700139,6700139,Pequenos Rebentos Ancestral Loureiro Pet Nat,2022,Portugal,Douro,Márcio Lopes,Vino espumoso,Loureiro,20.00,...,5.9,piedra,menta,menta,cítrico,pomelo,limón,piedra,tomillo,polvo de vainilla
3,https://www.vivino.com/ES/es/mestres-coquet-gran-reserva-brut-nature/w/4311421,4311421,Coquet Gran Reserva Brut Nature,2019,España,Cava,Mestres,Vino espumoso,"Macabeo, Xarel-lo, Parellada",15.87,...,6.0,pimienta verde,menta,No disponible,fresa madura,crema,uva pasa,roble,tostada,almizcle
4,https://www.vivino.com/ES/es/cipriani-bellini-cipriani/w/5794981,5794981,Bellini N.V.,No disponible,Italia,Vino d'Italia,Cipriani,Vino espumoso,No disponible,20.00,...,5.4,pomelo,solución salina,polvo de vainilla,hongo,granada,uva pasa,naranja,cítrico,pomelo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,https://www.vivino.com/ES/es/jacques-selosse-les-carelles-blanc-de-blancs-extra-brut-champagne-grand-cru-le-mesnil-sur-oger-lieux-dits/w/1101102,1101102,Les Carelles Blanc de Blancs Extra Brut Champagne Grand Cru 'Le Mesnil-sur-Oger' (Lieux Dits)N.V.,No disponible,Francia,"Champagne Grand Cru ""Le Mesnil-sur-Oger""",Jacques Selosse,Vino espumoso,No disponible,1112.00,...,6.7,minerales,minerales,humo,roble,tostada,tabaco,menta,tabaco dulce,uva pasa
4151,https://www.vivino.com/ES/es/agrapart-fils-exp-brut-nature-champagne/w/1455336,1455336,EXP. Brut Nature Champagne,2019,Francia,Champán,Agrapart & Fils,Vino espumoso,Chardonnay,342.91,...,7.2,menta,pimienta verde,No disponible,minerales,piedra,tiza,cítrico,limón,piedra
4152,https://www.vivino.com/ES/es/francis-orban-champagne-millesime-champagne/w/3979388,3979388,Millésimé Champagne,2016,Francia,Champán,Francis Orban,Vino espumoso,"Chardonnay, Pinot Meunier",101.18,...,7.3,cítrico,limón,limón,menta,pimienta verde,No disponible,roble,tabaco dulce,
4153,https://www.vivino.com/ES/es/duval-leroy-femme-de-champagne-brut-nature/w/11003457,11003457,Femme de Champagne Brut Nature,2002,Francia,Champagne Grand Cru,Duval-Leroy,Vino espumoso,"Chardonnay, Pinot Noir",279.99,...,6.9,tiza,minerales,terroso,tabaco dulce,pimienta verde,polvo de vainilla,roble,tostada,uva pasa


In [70]:
df_uvas_separado =  df["Uva"].str.split(',', expand=True)
df_uvas_separado["ID"]= df["ID"]
df_uvas_separado = df_uvas_separado.fillna("Desconocido")
df_uvas_separado[100:140]

,0,1,2,3,4,5,6,7,ID
100,Parellada,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,8763558
101,Macabeo,Xarel-lo,Parellada,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1154695
102,Timorasso,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,10909946
103,Asprinio Bianco,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1232522
104,No disponible,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1533757
105,Pinot Noir,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1957556
106,Parellada,Pinot Noir,Macabeo,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,1399465
107,Melon de Bourgogne,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,11440485
108,Pinot Noir,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,5906212
109,Pinot Blanc,Pinot Noir,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,Desconocido,2596568


In [71]:
df_uvas_separado = df_uvas_separado.drop(columns = "ID")

df_uvas_separado = df_uvas_separado.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_uvas_separado = pd.get_dummies(df_uvas_separado,drop_first=True ,prefix="" ,prefix_sep="")

# Convertir a DataFrame
df_uvas_separado = pd.DataFrame(df_uvas_separado)

# Mostrar resultado
display(df_uvas_separado)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/3747642572.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_uvas_separado = df_uvas_separado.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


,Airen,Albariño,Aleatico,Altesse,Alvarelhao,Alvarinho,Ancellotta,ArintodeBucelas,Arneis,AsprinioBianco,...,Tempranillo,Viura,Xarel-lo,Chardonnay,Desconocido,GrenacheBlanc,Malvasia,Welschriesling,Traminer,MoscatoGiallo
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4151,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4152,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4153,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [72]:
df_uvas_separado = df_uvas_separado.drop(columns="Desconocido")
df_uvas_separado=df_uvas_separado.groupby(axis=1, level=0).sum()
df_uvas_separado

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/1971516612.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_uvas_separado=df_uvas_separado.groupby(axis=1, level=0).sum()


,Airen,Albariño,Aleatico,Aligoté,Altesse,Alvarelhao,Alvarinho,Ancellotta,Arbane,ArintodeBucelas,...,Vinyater,Viognier,Viura,Weissburgunder,Welschriesling,Xarel-lo,Xinomavro,Zibibbo,Zinfandel,Zweigelt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df_uvas_separado.dtypes

Airen        int64
Albariño     int64
Aleatico     int64
Aligoté      int64
Altesse      int64
             ...  
Xarel-lo     int64
Xinomavro    int64
Zibibbo      int64
Zinfandel    int64
Zweigelt     int64
Length: 255, dtype: object

In [74]:
# Definir el umbral
umbral = 20

# Filtrar columnas cuya suma sea mayor al umbral
columnas_filtradas = df_uvas_separado.loc[:, df_uvas_separado.sum() > umbral]

columnas_filtradas

,Barbera,Chardonnay,CheninBlanc,Garnacha,Glera,Lambrusco,Macabeo,Malvasia,Moscato,MoscatoBianco,Nodisponible,Parellada,PinotBlanc,PinotMeunier,PinotNero,PinotNoir,Riesling,Trepat,Xarel-lo
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4151,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4152,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4153,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [75]:
df_uvas_separado_filtro = columnas_filtradas

In [76]:
df_uvas_separado_filtro["ID"] = df["ID"]

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/3015606806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uvas_separado_filtro["ID"] = df["ID"]


In [77]:
df_notas_sabor_mejorardo = df[["Taste_Notes_1_1","Taste_Notes_1_2","Taste_Notes_1_3","Taste_Notes_2_1","Taste_Notes_2_2","Taste_Notes_2_3","Taste_Notes_3_1","Taste_Notes_3_2","Taste_Notes_3_3","ID"]]


In [78]:
df_notas_sabor_mejorardo

,Taste_Notes_1_1,Taste_Notes_1_2,Taste_Notes_1_3,Taste_Notes_2_1,Taste_Notes_2_2,Taste_Notes_2_3,Taste_Notes_3_1,Taste_Notes_3_2,Taste_Notes_3_3,ID
0,menta,pomelo,mora,cítrico,limón,pomelo,fresa madura,crema,queso,3967508
1,crema,polvo de vainilla,polvo de vainilla,tabaco dulce,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,polvo de vainilla,4919479
2,piedra,menta,menta,cítrico,pomelo,limón,piedra,tomillo,polvo de vainilla,6700139
3,pimienta verde,menta,No disponible,fresa madura,crema,uva pasa,roble,tostada,almizcle,4311421
4,pomelo,solución salina,polvo de vainilla,hongo,granada,uva pasa,naranja,cítrico,pomelo,5794981
...,...,...,...,...,...,...,...,...,...,...
4150,minerales,minerales,humo,roble,tostada,tabaco,menta,tabaco dulce,uva pasa,1101102
4151,menta,pimienta verde,No disponible,minerales,piedra,tiza,cítrico,limón,piedra,1455336
4152,cítrico,limón,limón,menta,pimienta verde,No disponible,roble,tabaco dulce,,3979388
4153,tiza,minerales,terroso,tabaco dulce,pimienta verde,polvo de vainilla,roble,tostada,uva pasa,11003457


In [79]:
df_taste_1_separado_tijera_sin_ID = df_notas_sabor_mejorardo.drop(columns="ID")

In [80]:
valores_unicos_taste = set(df_taste_1_separado_tijera_sin_ID.values.ravel())

# Filtrar None y aplicar strip() solo a los strings
valores_unicos_taste = {s.strip() for s in valores_unicos_taste if isinstance(s, str)}

print(valores_unicos_taste)

{'', 'tofi', 'chocolate', 'carne adobada', 'vainilla', 'pimiento asado', 'algodón de azúcar', 'tostada', 'queso', 'cítrico', 'hongo', 'perfume', 'trufa', 'heno', 'eucalipto', 'bebida cola', 'tomillo', 'terroso', 'aceite balsámico', 'jengibre', 'banana', 'grafito', 'pimienta blanca', 'sandía', 'almendra', 'carne asada', 'tomate', 'peonía', 'hierbas secas', 'suelo del bosque', 'almizcle', 'hibisco', 'trufa negra', 'mermelada de bayas', 'frutos del bosque', 'pimienta verde', 'pomelo', 'polvo de granito', 'frutos rojos', 'canela', 'anís', 'piedra', 'pétalo de rosa', 'cereza roja', 'ciruela pasa', 'ciruela roja', 'aroma', 'solución salina', 'menta', 'caucho', 'mermelada', 'arándano rojo', 'mantequilla', 'frambuesa negra', 'granada', 'virutas', 'tarta de frutas', 'cereza', 'humo', 'nuez negra', 'regaliz', 'mulberry', 'hierbas', 'rosa seca', 'jalapeño', 'cera de abejas', 'especias exóticas', 'tiza', 'carne a la parrilla', 'salsa de fresa', 'cacao', 'ciruela', 'arándano', 'mora', 'crema', 'mer

In [81]:
# Convertir todo el DataFrame a One-Hot Encoding sin prefijos por columna
df_taste_1_one_hot_encoded = pd.get_dummies(df_taste_1_separado_tijera_sin_ID, prefix='', prefix_sep='')

# Convertir a DataFrame
df_taste_1_one_hot_encoded = pd.DataFrame(df_taste_1_one_hot_encoded)

df_taste_1_one_hot_encoded.columns= df_taste_1_one_hot_encoded.columns.str.strip()
# Mostrar resultado
display(df_taste_1_one_hot_encoded)

,aceite balsámico,acelga,ajedrea,almendra,almizcle,anís,arándano,arándano rojo,banana,bergamota,...,tiza,tofi,tomate,tomillo,tostada,trufa,trufa negra,uva,uva pasa,vainilla
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4151,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4152,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4153,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [82]:
df_taste_1_one_hot_encoded=df_taste_1_one_hot_encoded.groupby(axis=1, level=0).sum()
df_taste_1_one_hot_encoded

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/770153049.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_taste_1_one_hot_encoded=df_taste_1_one_hot_encoded.groupby(axis=1, level=0).sum()


,,No disponible,aceite balsámico,acelga,ajedrea,algodón de azúcar,almendra,almizcle,alquitrán,anís,...,tomate,tomillo,tostada,trufa,trufa negra,uva,uva pasa,vainilla,virutas,zarzamora
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4151,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4152,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4153,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [83]:
# Definir el umbral
umbral = 150

# Filtrar columnas cuya suma sea mayor al umbral
columnas_filtradas = df_taste_1_one_hot_encoded.loc[:, df_taste_1_one_hot_encoded.sum() > umbral]

columnas_filtradas

df_taste_1_one_hot_encoded = columnas_filtradas

In [84]:
df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.applymap(lambda x: 1 if x != 0 else 0)

/var/folders/79/fd_qzfk141g8svblsrdwccqr0000gn/T/ipykernel_6393/1271349127.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.applymap(lambda x: 1 if x != 0 else 0)


In [87]:
df_taste_1_one_hot_encoded = df_taste_1_one_hot_encoded.drop(columns="No disponible")

In [88]:
conteo_valores_unicos_taste = df_taste_1_one_hot_encoded.sum()

In [89]:
conteo_valores_unicos_taste

almendra              776
almizcle              233
banana                153
cereza                319
crema                 630
cítrico              2192
frambuesa             509
fresa                 624
fresa madura          620
frutos rojos          176
limón                1977
mantequilla           172
menta                2236
minerales            1021
mora                  416
naranja               291
piedra               1357
pimienta verde       1184
polvo de vainilla    2301
pomelo               1580
queso                 265
roble                1330
sal                   162
tabaco                264
tabaco dulce          367
tiza                  221
tostada              1152
vainilla              242
dtype: int64

In [90]:
df_taste_1_one_hot_encoded["ID"] = df["ID"]

In [91]:
df_merged_one_hot = pd.concat([df.set_index("ID"), df_maridaje_separado_encoded.set_index("ID"), df_taste_1_one_hot_encoded.set_index("ID"),df_uvas_separado_filtro.set_index("ID")], axis=1).reset_index()

display(df_merged_one_hot.head())

,ID,Url,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,MoscatoBianco,Nodisponible,Parellada,PinotBlanc,PinotMeunier,PinotNero,PinotNoir,Riesling,Trepat,Xarel-lo
0,3967508,https://www.vivino.com/ES/es/fr-simonnet-febvre-cremant-de-bourgogne-cuvee-s-brut/w/3967508,Crémant de Bourgogne Cuvée S Brut,2018,Francia,Crémant de Bourgogne,Simonnet-Febvre,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,0,0,0,0,0,0,1,0,0,0
1,4919479,https://www.vivino.com/ES/es/louis-loron-cremant-de-bourgogne-blanc-tradition/w/4919479,Crémant de Bourgogne Blanc TraditionN.V.,No disponible,Francia,Crémant de Bourgogne,Louis Loron,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,0,0,0,0,0,0,1,0,0,0
2,6700139,https://www.vivino.com/ES/es/marcio-lopes-pequenos-rebentos-ancestral-loureiro-pet-nat/w/6700139,Pequenos Rebentos Ancestral Loureiro Pet Nat,2022,Portugal,Douro,Márcio Lopes,Vino espumoso,Loureiro,20.00,...,0,0,0,0,0,0,0,0,0,0
3,4311421,https://www.vivino.com/ES/es/mestres-coquet-gran-reserva-brut-nature/w/4311421,Coquet Gran Reserva Brut Nature,2019,España,Cava,Mestres,Vino espumoso,"Macabeo, Xarel-lo, Parellada",15.87,...,0,0,1,0,0,0,0,0,0,1
4,5794981,https://www.vivino.com/ES/es/cipriani-bellini-cipriani/w/5794981,Bellini N.V.,No disponible,Italia,Vino d'Italia,Cipriani,Vino espumoso,No disponible,20.00,...,0,1,0,0,0,0,0,0,0,0


In [92]:
df_merged_one_hot = df_merged_one_hot.dropna(axis=0, how="any")

In [93]:
df_merged_one_hot

,ID,Url,Nombre,Año,País,Región,Bodega,Tipo de vino,Uva,Precio,...,MoscatoBianco,Nodisponible,Parellada,PinotBlanc,PinotMeunier,PinotNero,PinotNoir,Riesling,Trepat,Xarel-lo
0,3967508,https://www.vivino.com/ES/es/fr-simonnet-febvre-cremant-de-bourgogne-cuvee-s-brut/w/3967508,Crémant de Bourgogne Cuvée S Brut,2018,Francia,Crémant de Bourgogne,Simonnet-Febvre,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,0,0,0,0,0,0,1,0,0,0
1,4919479,https://www.vivino.com/ES/es/louis-loron-cremant-de-bourgogne-blanc-tradition/w/4919479,Crémant de Bourgogne Blanc TraditionN.V.,No disponible,Francia,Crémant de Bourgogne,Louis Loron,Vino espumoso,"Chardonnay, Pinot Noir",20.00,...,0,0,0,0,0,0,1,0,0,0
2,6700139,https://www.vivino.com/ES/es/marcio-lopes-pequenos-rebentos-ancestral-loureiro-pet-nat/w/6700139,Pequenos Rebentos Ancestral Loureiro Pet Nat,2022,Portugal,Douro,Márcio Lopes,Vino espumoso,Loureiro,20.00,...,0,0,0,0,0,0,0,0,0,0
3,4311421,https://www.vivino.com/ES/es/mestres-coquet-gran-reserva-brut-nature/w/4311421,Coquet Gran Reserva Brut Nature,2019,España,Cava,Mestres,Vino espumoso,"Macabeo, Xarel-lo, Parellada",15.87,...,0,0,1,0,0,0,0,0,0,1
4,5794981,https://www.vivino.com/ES/es/cipriani-bellini-cipriani/w/5794981,Bellini N.V.,No disponible,Italia,Vino d'Italia,Cipriani,Vino espumoso,No disponible,20.00,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,1101102,https://www.vivino.com/ES/es/jacques-selosse-les-carelles-blanc-de-blancs-extra-brut-champagne-grand-cru-le-mesnil-sur-oger-lieux-dits/w/1101102,Les Carelles Blanc de Blancs Extra Brut Champagne Grand Cru 'Le Mesnil-sur-Oger' (Lieux Dits)N.V.,No disponible,Francia,"Champagne Grand Cru ""Le Mesnil-sur-Oger""",Jacques Selosse,Vino espumoso,No disponible,1112.00,...,0,1,0,0,0,0,0,0,0,0
4151,1455336,https://www.vivino.com/ES/es/agrapart-fils-exp-brut-nature-champagne/w/1455336,EXP. Brut Nature Champagne,2019,Francia,Champán,Agrapart & Fils,Vino espumoso,Chardonnay,342.91,...,0,0,0,0,0,0,0,0,0,0
4152,3979388,https://www.vivino.com/ES/es/francis-orban-champagne-millesime-champagne/w/3979388,Millésimé Champagne,2016,Francia,Champán,Francis Orban,Vino espumoso,"Chardonnay, Pinot Meunier",101.18,...,0,0,0,0,1,0,0,0,0,0
4153,11003457,https://www.vivino.com/ES/es/duval-leroy-femme-de-champagne-brut-nature/w/11003457,Femme de Champagne Brut Nature,2002,Francia,Champagne Grand Cru,Duval-Leroy,Vino espumoso,"Chardonnay, Pinot Noir",279.99,...,0,0,0,0,0,0,1,0,0,0


In [94]:
df_merged_one_hot["ID"] = df_merged_one_hot["ID"].astype(int)

In [95]:
df_merged_one_hot = df_merged_one_hot.drop(columns=["Tipo de vino","Url"])

In [97]:
df_merged_one_hot[["Nombre","País","Región","Bodega"]] = df_merged_one_hot[["Nombre","País","Región","Bodega"]].astype(str)

In [98]:
df_merged_one_hot_droped= df_merged_one_hot.drop(columns=["Año","Uva","Nombre","Maridajes","Taste_Notes_1_1","Taste_Notes_1_2","Taste_Notes_1_3","Taste_Notes_2_1","Taste_Notes_2_2","Taste_Notes_2_3","Taste_Notes_3_1","Taste_Notes_3_2","Taste_Notes_3_3"])

In [99]:
df_merged_one_hot_droped

,ID,País,Región,Bodega,Precio,Valoración,Contenido de alcohol,Ligero/Poderoso,Débil/Ácido,Amable/Con Burbujas,...,MoscatoBianco,Nodisponible,Parellada,PinotBlanc,PinotMeunier,PinotNero,PinotNoir,Riesling,Trepat,Xarel-lo
0,3967508,Francia,Crémant de Bourgogne,Simonnet-Febvre,20.00,3.7,12,4.6,5.6,6.0,...,0,0,0,0,0,0,1,0,0,0
1,4919479,Francia,Crémant de Bourgogne,Louis Loron,20.00,4.0,No disponible,5.4,5.2,6.5,...,0,0,0,0,0,0,1,0,0,0
2,6700139,Portugal,Douro,Márcio Lopes,20.00,4.1,No disponible,4.0,7.3,5.9,...,0,0,0,0,0,0,0,0,0,0
3,4311421,España,Cava,Mestres,15.87,3.9,2,4.2,5.5,6.0,...,0,0,1,0,0,0,0,0,0,1
4,5794981,Italia,Vino d'Italia,Cipriani,20.00,4.2,5,2.9,4.0,5.4,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4150,1101102,Francia,"Champagne Grand Cru ""Le Mesnil-sur-Oger""",Jacques Selosse,1112.00,4.6,5,6.7,7.2,6.7,...,0,1,0,0,0,0,0,0,0,0
4151,1455336,Francia,Champán,Agrapart & Fils,342.91,4.3,13,5.5,7.8,7.2,...,0,0,0,0,0,0,0,0,0,0
4152,3979388,Francia,Champán,Francis Orban,101.18,4.0,12,6.5,7.4,7.3,...,0,0,0,0,1,0,0,0,0,0
4153,11003457,Francia,Champagne Grand Cru,Duval-Leroy,279.99,4.4,12,5.3,7.0,6.9,...,0,0,0,0,0,0,1,0,0,0


In [100]:
df_merged_one_hot_droped.to_csv("df_mergedf_espumosos.csv")